# 753 Cracking the Safe

简单说，给出 n、k。n 个、k 种数字任意排列组合的所有方式…

构造一个最短的字符串（之一就好了），使得所有可能的排列方式都是该字符串的一个连续子串。

n < 4，k < 10。又是这种小数据输入的难题…

Alex Wice 的两个解法，一个是 Hierholzer 算法，一个是逆 Burrows-Wheeler 变换法。

当然，他的解法必须搭配评论吃。该丢他回中学重修语文。

## Hierholzer's Algorithm

把我们的问题抽象成这样：一个有向图中存在 k^(n-1) 个节点（每个节点的值对应长度为 n - 1 的全部可能密钥），每个节点都有 k 条出边（可以用的 k 个数字），连接到 (node.val + str(i))[1:] 这个节点，也就是选择了数字 i 之後会进入的状态。

例如，k = 2，n = 3 时，我们的图应该这么构造：

00 -[0]->00
00 -[1]->01
10->[0]->00
01->[0]->10

每一个节点加上其具有的一条边拼接成一个 n 位（k 进制）的数字，我们把这一组数据当作一个完整的密钥。然后，我们只需要找到一条欧拉路径即可。欧拉路径会经过所有的边一次、且仅一次。

注意这里欧拉路径必然存在：因为每个节点的出度都是 k（我们就是这么构造的），而每个节点的入度也是 k（因为对于  node k 来说，所有的入度值都一定是 k.val 的最後一位，而被丢弃的第一位则有 k 种可能（因为我们是 k 进制啊！）），所以每个节点的总度数一定为 2k，偶数。

转换成这种思路之後，我们会发现实际上，对于任何一个给定的 n、k，这种 Cracking Safe 的最短长度是一定的，可以直接算出来的。

问题变成找欧拉路径之后，问题就简单了。

遍历完之后，把它们 join 起来得到的就是这个万能密码。

代码简单到不可信：

In [ ]:
class Solution:
    def crackSafe(self, n: int, k: int) -> str:
        seen = set()
        ans = []
        def dfs(node):
            for x in map(str, range(k)):
                nei = node + x
                if nei not in seen:
                    seen.add(nei)
                    dfs(nei[1:])
                    ans.append(x)

        dfs("0" * (n-1))
        return "".join(ans) + "0" * (n-1)

这种方法的复杂度已经到了 O（n * k ^ n）。

但如果我们把所有 k^n 个密码做排列、每一个都检查一遍他们首尾之间的公共序列长度（也就是可以省下来的那一点长度），消耗的总时间会是 O（(n * k ^ n)!）。这就是魔鬼了。

破题。